In [19]:
import os
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math

In [20]:
#number of atoms
natoms = 9
nfreqs = 21
allfreqs = 21#int((nfreqs**2+nfreqs*3)/2)

In [21]:
#Read structure in A and atoms from Gaussian input file
with open('freq.gjf', 'r') as f:
    lines = f.readlines()
struc_A_IM = lines[-natoms:]
atoms = np.zeros(natoms,str)
struc_A = np.zeros((natoms,3))
for nr,i in enumerate(struc_A_IM):
    atoms[nr] = i.split()[0]
    struc_A[nr] = i.split()[1:]

In [22]:
#Read structure in A from Gaussian input file
with open('freq_td_root3.fch', 'r') as f:
    for line in f:
        if 'Current cartesian coordinates' in line:
            struc_au_IM = f.readlines()[:math.ceil(natoms*3 / 5)]
            break

struc_au = []
for i in struc_au_IM:
    struc_au = struc_au + i.split()
struc_au = np.array(struc_au,dtype=float).reshape(natoms,3)

In [23]:
#Read vibrational energies
with open('freq_td_root3.fch', 'r') as f:
    for line in f:
        if 'Vib-E2' in line:
            data_IM = f.readlines()[:math.ceil( allfreqs*4 / 5)]
            break

data = []
energies = []
IR = []
for i in data_IM:
    data = data + i.split()
energies = np.array(data[:nfreqs],dtype=float)
IR = np.array(data[allfreqs*3:(allfreqs*3+nfreqs)],dtype=float)

In [24]:
#Read harmonic modes
with open('freq_td_root3.fch', 'r') as f:
    for line in f:
        if 'Vib-Modes' in line:
            data_IM = f.readlines()[:math.ceil( nfreqs*natoms*3 / 5)]
            break
data = []
for i in data_IM:
    data = data + i.split()
data = data[:nfreqs*natoms*3]
modes = np.array(data,dtype=float).reshape(nfreqs,natoms,3)

In [25]:
#sortlist = np.argsort(energies)
sortlist = np.arange(nfreqs)

In [26]:
with open ('freq.molden', 'w') as f:
    f.write('[Molden Format]\n')
    f.write('[GEOMETRIES] XYZ\n')
    f.write('%5i' %  natoms+'\n')
    f.write('\n')
    for i in range(natoms):
        f.write(' ' + atoms[i] + '    ' + "%9.6f" % struc_A[i,0] + '    ' + "%9.6f" % struc_A[i,1] +'    ' +  "%9.6f" % struc_A[i,2] + '\n' )
    f.write('[FREQ]\n')
    for i in sortlist:#range(nfreqs):
        f.write('%10.4f' % energies[i] + '\n')
    f.write('[INT]\n')
    for i in sortlist:#range(nfreqs):
        f.write('%10.4f' % IR[i] + '     0.0000\n')
    f.write('[FR-COORD]\n')
    for i in range(natoms):
        f.write(' ' + atoms[i] + '    ' + "%9.6f" % struc_au[i,0] + '    ' + "%9.6f" % struc_au[i,1] +'    ' +  "%9.6f" % struc_au[i,2] + '\n' )
    f.write('[FR-NORM-COORD]\n')
    for nr,i in enumerate(sortlist):#range(nfreqs):
        f.write('vibration ' + '%5i' % (nr+1) + '\n')
        for j in range(natoms):
            f.write("%12.6f" % modes[i,j,0] + ' ' + "%12.6f" % modes[i,j,1] + ' ' +  "%12.6f" % modes[i,j,2] + '\n' )

In [27]:
np.shape(modes)

(21, 9, 3)

In [28]:
mass_c = 12.000000 
mass_o = 15.994915 
mass_h = 1.007825 
molecule = [mass_c] * 4 + [mass_o] + [mass_h] * 4

In [29]:
molecule

[12.0, 12.0, 12.0, 12.0, 15.994915, 1.007825, 1.007825, 1.007825, 1.007825]

In [30]:
norm = np.zeros(nfreqs)
for imode in range(nfreqs):
    norm[imode] = 0.0
    #calculate norm
    for j, atom in enumerate(molecule):
        for xyz in range(3):
            norm[imode] +=  modes[imode,j,xyz]**2*atom
    norm[imode] = math.sqrt(norm[imode])
    #print(norm[imode])
print(norm)

[1.58059842 1.60079919 2.84124311 1.09205976 2.19800607 1.25497848
 1.21890291 1.12728078 1.26460453 2.28383311 1.15179725 1.19367177
 2.20993874 1.10646412 1.57763404 2.02246704 1.77767981 1.04905673
 1.05152824 1.04679796 1.05419475]


In [31]:
for imode in range(nfreqs):
    for j, atom in enumerate(molecule):
        for xyz in range(3):
            modes[imode,j,xyz] /= norm[imode]/math.sqrt(atom)

In [32]:
results = np.transpose(modes.reshape(nfreqs,27))

In [33]:
np.dot(results.T,results)

array([[ 1.00000000e+00, -1.02296504e-10,  2.00658685e-10,
         9.83238515e-09, -1.11098532e-11,  1.72013367e-08,
        -1.38293744e-10, -1.41510866e-10,  5.09692071e-12,
         7.74678339e-12, -3.27408511e-14,  4.66074750e-13,
        -3.04130805e-13,  1.27938039e-12, -8.14202910e-13,
        -1.89765051e-12, -9.56496339e-14,  1.99464283e-12,
        -8.65863825e-13,  1.52573177e-14, -1.76334189e-13],
       [-1.02296504e-10,  1.00000000e+00, -1.01456777e-11,
        -2.80687312e-10,  7.92446094e-12, -8.91977309e-10,
         1.14023784e-08, -1.43848947e-08,  1.94316921e-12,
         3.84960437e-12,  3.30299040e-12, -3.03063252e-13,
         2.91061359e-12, -1.08448897e-12,  2.08857354e-13,
        -1.04984194e-12,  1.24413009e-13,  1.21406492e-12,
        -1.14228520e-12, -3.18897231e-13, -2.55370974e-13],
       [ 2.00658685e-10, -1.01456777e-11,  1.00000000e+00,
        -6.61266548e-10,  2.42034627e-10,  7.84234389e-11,
        -4.34430157e-12, -6.23021685e-12,  3.39022497e

In [34]:
import pandas
from pandas import DataFrame
print(DataFrame(np.dot(results.T,results)))

              0             1             2             3             4   \
0   1.000000e+00 -1.022965e-10  2.006587e-10  9.832385e-09 -1.110985e-11   
1  -1.022965e-10  1.000000e+00 -1.014568e-11 -2.806873e-10  7.924461e-12   
2   2.006587e-10 -1.014568e-11  1.000000e+00 -6.612665e-10  2.420346e-10   
3   9.832385e-09 -2.806873e-10 -6.612665e-10  1.000000e+00 -1.896090e-11   
4  -1.110985e-11  7.924461e-12  2.420346e-10 -1.896090e-11  1.000000e+00   
5   1.720134e-08 -8.919773e-10  7.842344e-11  3.503030e-09  1.728494e-11   
6  -1.382937e-10  1.140238e-08 -4.344302e-12  6.091818e-11 -2.046042e-11   
7  -1.415109e-10 -1.438489e-08 -6.230217e-12  4.189524e-10 -4.392224e-12   
8   5.096921e-12  1.943169e-12  3.390225e-09 -6.336424e-11  3.277034e-10   
9   7.746783e-12  3.849604e-12  5.291083e-09 -1.166012e-10 -6.476024e-11   
10 -3.274085e-14  3.302990e-12 -1.537720e-10  2.745493e-12  1.039249e-08   
11  4.660747e-13 -3.030633e-13 -2.244747e-10 -8.617460e-12  8.211561e-09   
12 -3.041308